# DMML Assignment 2
### ===========================================
### Name: Aishee Dasgupta, Roll No. : MDS201815 
### Name: Trina De, Roll No. : MDS201824


## 1. First we load all the packages we might need to run our code

In [1]:
## loading packages
import pandas as pd
import numpy as np
import codecs
import re
from glob import iglob
from bs4 import BeautifulSoup
from skmultilearn.problem_transform import LabelPowerset
from sklearn.preprocessing import MultiLabelBinarizer
import time
import csv
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
Data = pd.DataFrame()

### 2. We then extract everything non-numerical from the Body and Title tags of the documents into a dataframe with two columns named Body and Topics

In [3]:
start=time.clock() ## starting to calculate data processing time
invalid = re.compile(r"&#(\d|[0-2]\d| 3[01])", re.I | re.M) ## Removing all characters with ASCII code below 
for file in iglob("F:\CMI\Sem_2\Machine learning\Assignment_work\Reuters Data\*.sgm"): ## iterating through files in folder
    with codecs.open(file, encoding='utf-8',errors='ignore') as fo:
        sgml=invalid.sub('', fo.read()) ## Substituting such characters found in invalid by blanks
    soup = BeautifulSoup(sgml,'html.parser')

    Topics = []
    Body = []
    reuters = soup.find_all('reuters') ## Iterating over files
    for rts in reuters:
        if rts.body: ## If article has a body
            title = rts.find('text').title.string.lower()
            body = rts.find('text').body.string.lower()
            bdy = title + " " + body
            spc = re.compile(r"[\t\n]+")
            bdy = spc.sub(' ', bdy)
            if not rts.topics.find('d'): ## If no topics are assigned
                Topics.append(None)
                Body.append(bdy)
            else: ## If topics are assigned
                temp = []
                for top in rts.topics.find_all('d'):
                    temp.append(str(top.text))
                Topics.append(temp)
                Body.append(bdy)
                    
        else:
            if rts.title: ## It there is a title
                bdy = rts.find('text').title.string.lower()
                spc = re.compile(r"[\t\n]+")
                if not rts.topics.find('d'): ## If there is no topics assigned
                    Topics.append(None)
                    Body.append(bdy)
                else: ## If topic is assigned
                    temp = []
                    for top in rts.topics.find_all('d'):
                        temp.append(str(top.text))
                    Topics.append(temp)
                    Body.append(bdy)
            else: ## If there is no title
                bdy = rts.find('text').string.lower()
                spc = re.compile(r"[\t\n]+")
                if not rts.topics.find('d'): ## If there is no topics assigned
                    Topics.append(None)
                    Body.append(bdy)
                else: ## If topic is assigned
                    temp = []
                    for top in rts.topics.find_all('d'):
                        temp.append(str(top.text))
                    Topics.append(temp)
                    Body.append(bdy)
         
    d= {'Body' : Body,'Topics' : Topics} 
    Data = pd.concat([Data, pd.DataFrame(d)]) ## creating the dataframe
end=time.clock() ## stopping calculation of data processing time
data_processing_time = end-start
data_processing_time

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


25.797221600000004

In [4]:
Data.head()

,Body,Topics
0,bahia cocoa review showers continued throughou...,[cocoa]
1,standard oil <srd> to form financial unit stan...,None
2,texas commerce bancshares <tcb> files plan tex...,None
3,talking point/bankamerica <bac> equity offer b...,None
4,national average prices for farmer-owned reser...,"[grain, wheat, corn, barley, oat, sorghum]"


### 3. We are dropping  those rows out of the dataframe whose topics is null

In [5]:
DataFil = Data[Data['Topics'].notnull()] ## taking only columns were topic is not null 
DataFil.describe()

,Body,Topics
count,11367,11367
unique,11233,655
top,bundesbank leaves credit policies unchanged th...,[earn]
freq,4,3945


### 4. We drop duplicate rows out of the dataframe.

In [6]:
DataFil.drop_duplicates(subset = "Body" ,inplace = True) ## dropping duplicates of (document, topic) sets

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
DataFil.describe()

,Body,Topics
count,11233,11233
unique,11233,655
top,harris bank increases its prime rate <the harr...,[earn]
freq,1,3937


### 5. Creating a sparse matrix for the topics

In [8]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(DataFil['Topics']) ## creating the sparse matrix for the topics assigned against various documents

### 6. Training the model after vectorizing taking only the words out of the document
### Using ComplementNB as the classifier and setting the smoothing factor alpha at 0.8

In [43]:
start2=time.clock() ## starting to calculate the model training time
X_train, X_test, y_train, y_test = train_test_split( DataFil['Body'] , y , random_state = 0)
tfidf_vect = TfidfVectorizer(token_pattern = r'\b[^\d\W]+\b',stop_words = text.ENGLISH_STOP_WORDS) ## removing digits and using standard english language words as stop words
tfidf_vect.fit(X_train)
X_train_tfidf=tfidf_vect.transform(X_train) ## vectorizing train documents
classifier = LabelPowerset(ComplementNB(alpha=0.8)) ## defining the classifier
classifier.fit(X_train_tfidf,y_train) ## training the classifier with the training data
X_test_tfidf=tfidf_vect.transform(X_test) ## vectorizing test documents
end2=time.clock() ## stopping calculation of model training time
model_training_time=end2-start2
model_training_time 

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  if __name__ == '__main__':


10.422734899999796

In [10]:
pred = classifier.predict(X_test_tfidf) ## predicting topics for test documents

In [28]:
Output = pd.DataFrame({'test data' : X_test , 'actual topic': mlb.inverse_transform(y_test), 'predicted topic' : mlb.inverse_transform(pred.toarray())})
Output = Output.reset_index(level=None, drop=True, inplace=False, col_level=0, col_fill='')

Output.to_csv("F:\CMI\Sem_2\Machine learning\Assignment_work\Text_classification_output.csv") ## writing output to file
with open("F:\CMI\Sem_2\Machine learning\Assignment_work\Text_classification_output.csv",'a') as fd: ## opening file
    row1=["The time taken to process the data is",data_processing_time] ## writing data processing time
    row2=["The time taken to train is",model_training_time] ## writing model training time
    writer = csv.writer(fd)
    writer.writerow(row1)
    writer.writerow(row2)

Output.head()

,test data,actual topic,predicted topic
0,japan warns u.s. it may retaliate in trade dis...,"(trade,)","(trade,)"
1,amertek inc <atekf> 1st qtr net shr profit 20 ...,"(earn,)","(earn,)"
2,zayre corp <zy> 4th qtr jan 31 net shr 73 cts ...,"(earn,)","(earn,)"
3,<first montauk securities> in merger deal firs...,"(acq,)","(acq,)"
4,spain cargo firms hire dockers to offset strik...,"(ship,)","(ship,)"


### 7. Calculating accuracy measures

In [29]:
print(accuracy_score(y_test,pred.toarray()))

0.797436810252759


In [30]:
fscore = f1_score(y_test,pred.toarray(),average='weighted')
print(f1_score(y_test,pred.toarray(),average='weighted'))

0.7843659945925405


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [31]:
print(recall_score(y_test,pred.toarray(),average='weighted'))
recall = recall_score(y_test,pred.toarray(),average='weighted')

0.7646231721034871


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [32]:
print(precision_score(y_test,pred.toarray(),average='weighted'))
precision = precision_score(y_test,pred.toarray(),average='weighted')

0.8597417251933084


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
row1=["The Precision is:",fscore]
row2=["The recall is:",recall]
row3=["The fscore is:",fscore]
with open("F:\CMI\Sem_2\Machine learning\Assignment_work\Text_classification_output.csv",'a') as fd: ## writing measures to file
    row=["The time taken to train is",data_processing_time]
    writer = csv.writer(fd)
    writer.writerow(row1)
    writer.writerow(row2)
    writer.writerow(row3)

In [34]:
mlb.inverse_transform(pred.toarray()) ## mapping sparse arary output to topics

[('trade',),
 ('earn',),
 ('earn',),
 ('acq',),
 ('ship',),
 ('earn',),
 ('acq',),
 ('acq',),
 ('acq',),
 ('acq',),
 ('earn',),
 ('sugar',),
 ('orange',),
 ('earn',),
 ('coffee',),
 ('acq',),
 ('earn',),
 ('interest', 'money-fx'),
 ('acq',),
 ('acq',),
 ('acq',),
 ('earn',),
 ('interest', 'money-fx'),
 ('money-fx',),
 ('trade',),
 ('earn',),
 ('acq',),
 ('earn',),
 ('earn',),
 ('earn',),
 ('earn',),
 ('acq',),
 ('acq',),
 ('acq',),
 ('grain', 'wheat'),
 ('trade',),
 ('interest', 'money-fx'),
 ('earn',),
 ('housing',),
 ('earn',),
 ('copper', 'gold', 'nickel', 'palladium', 'platinum'),
 ('earn',),
 ('earn',),
 ('earn',),
 ('gold',),
 ('acq',),
 ('acq',),
 ('ship',),
 ('jobs',),
 ('acq',),
 ('earn',),
 ('earn',),
 ('earn',),
 ('crude',),
 ('acq',),
 ('earn',),
 ('acq',),
 ('earn',),
 ('dlr', 'money-fx'),
 ('coffee',),
 ('trade',),
 ('earn',),
 ('earn',),
 ('trade',),
 ('acq',),
 ('iron-steel',),
 ('acq',),
 ('corn', 'grain'),
 ('crude',),
 ('acq',),
 ('copper',),
 ('earn',),
 ('interest'

In [35]:
for item in mlb.inverse_transform(pred.toarray()): ## making sure of no null predictions
    if item == None:
        print(item)

### 8. Predicting for those documents that was not assigned any topics in the original data

In [36]:
Data2 = Data[Data['Topics'].isnull()] ## extracting data with topics as null

In [37]:
Data2.head()

,Body,Topics
1,standard oil <srd> to form financial unit stan...,None
2,texas commerce bancshares <tcb> files plan tex...,None
3,talking point/bankamerica <bac> equity offer b...,None
6,red lion inns files plans offering red lion in...,None
7,usx <x> debt dowgraded by moody's moody's inve...,None


In [38]:
X_test2_tfidf=tfidf_transformer.transform(Data2['Body'])

In [39]:
pred2 = classifier.predict(X_test2_tfidf) ## predicting topics which were originally null

In [40]:
len(mlb.inverse_transform(pred2.toarray()))

10211

In [41]:
Output2 = pd.DataFrame({'test data' : Data2['Body'] , 'predicted topic' : mlb.inverse_transform(pred2.toarray())})
Output2 = Output2.reset_index(level=None, drop=True, inplace=False, col_level=0, col_fill='')

Output2.to_csv("F:\CMI\Sem_2\Machine learning\Assignment_work\Text_classification_output2.csv") ## writing output to file
Output2.head()

,test data,predicted topic
0,standard oil <srd> to form financial unit stan...,"(acq,)"
1,texas commerce bancshares <tcb> files plan tex...,"(acq,)"
2,talking point/bankamerica <bac> equity offer b...,"(acq,)"
3,red lion inns files plans offering red lion in...,"(acq,)"
4,usx <x> debt dowgraded by moody's moody's inve...,"(acq,)"


In [42]:
Output2.describe()

,test data,predicted topic
count,10211,10211
unique,10056,96
top,japan likely to allow foreign futures market u...,"(acq,)"
freq,3,5201
